In [10]:
import numpy as np
import pandas as pd
import warnings
import random
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler, PowerTransformer, StandardScaler
from sklearn.random_projection import GaussianRandomProjection
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import log_loss, accuracy_score
from scipy.optimize import minimize
from scipy.fft import fft
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

# GPU CHECK
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("✅ GPU DETECTED: HRF v26.0 'Holo-Fractal Universe' Active")
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ GPU NOT FOUND: Running in Slow Mode")

warnings.filterwarnings('ignore')

# --- 1. THE HOLOGRAPHIC SOUL (Unit 3 - Multiverse Edition) ---
class HolographicSoulUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, k=15):
        self.k = k
        # DNA now defines a 'Universe' of physics
        self.dna_ = {
            'freq': 2.0,
            'gamma': 0.5,
            'power': 2.0,
            'metric': 'minkowski', # Generalized Space
            'p': 2.0,              # The curvature of space (2=Euclidean, 1=Manhattan)
            'phase': 0.0,          # The shift of the wave
            'dim_reduction': 'none'
        }
        self.projector_ = None
        self.X_raw_source_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        # Apply Projection based on current DNA
        self._apply_projection(X)
        self.y_train_ = y
        return self

    def _apply_projection(self, X):
        # "Lens" adjustment
        if self.dna_['dim_reduction'] == 'holo':
            n_components = max(2, int(np.sqrt(X.shape[1])))
            self.projector_ = GaussianRandomProjection(n_components=n_components, random_state=42)
            self.X_train_ = self.projector_.fit_transform(X)
        elif self.dna_['dim_reduction'] == 'pca':
             n_components = max(2, int(np.sqrt(X.shape[1])))
             self.projector_ = PCA(n_components=n_components, random_state=42)
             self.X_train_ = self.projector_.fit_transform(X)
        else:
            self.projector_ = None
            self.X_train_ = X

    def evolve(self, X_val, y_val, generations=1000):
        # "Multiverse" Evolution Strategy
        # Instead of 1 mutation, we spawn 'n_universes' and keep the best
        n_universes = 10
        best_acc = self.score(X_val, y_val)
        best_dna = self.dna_.copy()

        # 1. Smart Initialization (The Big Bang)
        if GPU_AVAILABLE:
            sample_X = cp.asarray(self.X_train_[:100])
            # Use standard euclidean for initial guess to avoid complexity
            dists = cp.mean(cp.linalg.norm(sample_X[:, None, :] - sample_X[None, :, :], axis=2))
            median_dist = float(cp.asnumpy(dists))
        else:
            median_dist = 1.0

        if median_dist > 0:
            best_dna['freq'] = 3.14159 / median_dist

        for i in range(generations):
            candidates = []

            # Spawn Parallel Universes (Mutations)
            for _ in range(n_universes):
                mutant = best_dna.copy()
                trait = random.choice(list(mutant.keys()))

                # "Cosmic Inflation" - Mutation Logic
                if trait == 'freq':
                    mutant['freq'] *= np.random.uniform(0.8, 1.25)
                elif trait == 'gamma':
                    mutant['gamma'] = np.random.uniform(0.1, 5.0)
                elif trait == 'power':
                    mutant['power'] = random.choice([0.5, 1.0, 2.0, 3.0, 4.0, 6.0]) # Expanded range
                elif trait == 'p':
                    # Evolve the geometry of space itself (0.5 to 5.0)
                    mutant['p'] = np.clip(mutant['p'] + np.random.uniform(-0.5, 0.5), 0.5, 8.0)
                elif trait == 'phase':
                    mutant['phase'] = np.random.uniform(0, 3.14159) # Shift the wave
                elif trait == 'dim_reduction':
                    mutant['dim_reduction'] = random.choice(['none', 'holo', 'pca'])

                candidates.append(mutant)

            # Evaluate Universes
            generation_best_acc = -1
            generation_best_dna = None

            for mutant_dna in candidates:
                self.dna_ = mutant_dna

                # If projection changed, we must re-project the training data
                # (We use the raw source stored in self.X_raw_source_)
                if self.X_raw_source_ is not None:
                     self._apply_projection(self.X_raw_source_)

                acc = self.score(X_val, y_val)
                if acc > generation_best_acc:
                    generation_best_acc = acc
                    generation_best_dna = mutant_dna

            # Survival of the Fittest Universe
            if generation_best_acc >= best_acc:
                best_acc = generation_best_acc
                best_dna = generation_best_dna
            else:
                # Revert to the last known stable universe
                self.dna_ = best_dna
                if self.X_raw_source_ is not None:
                    self._apply_projection(self.X_raw_source_)

        self.dna_ = best_dna
        return best_acc

    def set_raw_source(self, X):
        self.X_raw_source_ = X

    def predict_proba(self, X):
        if self.projector_ is not None:
            X_curr = self.projector_.transform(X)
        else:
            X_curr = X

        if GPU_AVAILABLE:
            return self._predict_proba_gpu(X_curr)
        else:
            return np.zeros((len(X), len(self.classes_)))

    def _predict_proba_gpu(self, X):
        X_tr_g = cp.asarray(self.X_train_, dtype=cp.float32)
        X_te_g = cp.asarray(X, dtype=cp.float32)
        y_tr_g = cp.asarray(self.y_train_)

        n_test = len(X_te_g)
        n_classes = len(self.classes_)
        probas = []
        batch_size = 256


        # Extract Physics Constants
        p_norm = self.dna_.get('p', 2.0)
        gamma = self.dna_['gamma']
        freq = self.dna_['freq']
        power = self.dna_['power']
        phase = self.dna_.get('phase', 0.0)

        for i in range(0, n_test, batch_size):
            end = min(i + batch_size, n_test)
            batch_te = X_te_g[i:end]

            # 1. Generalized Minkowski Distance (The Shape of Space)
            # dist = sum(|x - y|^p)^(1/p)
            diff = cp.abs(batch_te[:, None, :] - X_tr_g[None, :, :])
            dists = cp.sum(cp.power(diff, p_norm), axis=2)
            dists = cp.power(dists, 1.0/p_norm)

            # Top K Nearest Neighbors in this specific spacetime
            top_k_idx = cp.argsort(dists, axis=1)[:, :self.k]
            row_idx = cp.arange(len(batch_te))[:, None]
            top_dists = dists[row_idx, top_k_idx]
            top_y = y_tr_g[top_k_idx]

            # 2. THE FINAL HRF WAVE EQUATION
            # w = e^(-gamma * d^2) * (1 + cos(freq * d + phase)) ^ power
            w = cp.exp(-gamma * (top_dists**2)) * (1.0 + cp.cos(freq * top_dists + phase))
            w = cp.power(w, power)

            batch_probs = cp.zeros((len(batch_te), n_classes))
            for c_idx, cls in enumerate(self.classes_):
                class_mask = (top_y == cls)
                batch_probs[:, c_idx] = cp.sum(w * class_mask, axis=1)

            total_energy = cp.sum(batch_probs, axis=1, keepdims=True)
            total_energy[total_energy == 0] = 1.0
            batch_probs /= total_energy
            probas.append(batch_probs)

            # Clean up VRAM
            del batch_te, dists, diff, top_k_idx, top_dists, w
            cp.get_default_memory_pool().free_all_blocks()

        return cp.asnumpy(cp.concatenate(probas))

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]

    def score(self, X, y):
        return accuracy_score(y, self.predict(X))


# --- 2. G.O.D. v26 (The Holographic Manager) ---
class HarmonicResonanceClassifier_GOD_v26(BaseEstimator, ClassifierMixin):
    def __init__(self, verbose=False):
        self.verbose = verbose
        # We start with simple scaling, let the Soul handle the projection
        self.scaler_ = RobustScaler(quantile_range=(10.0, 90.0))
        self.unit_logic = ExtraTreesClassifier(n_jobs=-1, random_state=42)
        self.unit_grad = XGBClassifier(use_label_encoder=False, eval_metric='logloss',
                                       tree_method='hist', n_jobs=-1, random_state=42)
        self.unit_soul = HolographicSoulUnit(k=15)
        self.weights_ = None

    def _apply_physics(self, X):
        # Physics Engine: Only add FFT if dimensions suggest time-series/sequence
        # For pure images (Optdigits), diffs might be noise, so we rely on the Holographic Projection inside Soul
        return X

    def fit(self, X, y):
        y = np.array(y).astype(int)
        X, y = check_X_y(X, y)
        self.classes_ = np.unique(y)

        # Scale
        X_scaled = self.scaler_.fit_transform(X)

        # Split for Evolution
        X_evo_t, X_evo_v, y_evo_t, y_evo_v = train_test_split(
            X_scaled, y, test_size=0.2, stratify=y, random_state=42
        )

        if self.verbose:
            print("  > Initiating G.O.D. v26 (Holo-Fractal Universe)...")

        # --- EVOLUTION PHASE ---
        # A. Logic
        best_logic_score, best_logic_params = 0, {}
        for _ in range(3):  # reduced iterations for speed
            params = {
                'n_estimators': random.randint(100, 300),
                'max_features': random.choice(['sqrt', 'log2', 0.5]),
                'min_samples_split': random.randint(2, 6),
                'bootstrap': False
            }
            clf = ExtraTreesClassifier(**params, n_jobs=-1, random_state=42)
            clf.fit(X_evo_t, y_evo_t)
            s = clf.score(X_evo_v, y_evo_v)
            if s > best_logic_score:
                best_logic_score, best_logic_params = s, params
        self.unit_logic.set_params(**best_logic_params)

        # B. Gradient
        best_grad_score, best_grad_params = 0, {}
        for _ in range(3):
            params = {
                'n_estimators': random.randint(100, 300),
                'learning_rate': np.random.uniform(0.05, 0.2),
                'max_depth': random.randint(4, 8),
                'subsample': np.random.uniform(0.6, 1.0)
            }
            clf = XGBClassifier(**params, use_label_encoder=False,
                                eval_metric='logloss', tree_method='hist',
                                n_jobs=-1, random_state=42)
            clf.fit(X_evo_t, y_evo_t)
            s = clf.score(X_evo_v, y_evo_v)
            if s > best_grad_score:
                best_grad_score, best_grad_params = s, params
        self.unit_grad.set_params(**best_grad_params)

        # C. Soul (Holographic Evolution)
        self.unit_soul.set_raw_source(X_evo_t)
        self.unit_soul.fit(X_evo_t, y_evo_t)
        soul_score = self.unit_soul.evolve(X_evo_v, y_evo_v, generations=20)

        if self.verbose:
            print(f"    [Evolution] Logic: {best_logic_score:.2%} | "
                  f"Grad: {best_grad_score:.2%} | Soul: {soul_score:.2%}")

        # --- STACKING PHASE ---
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        oof_brain = np.zeros((len(X), len(self.classes_)))
        oof_grad = np.zeros((len(X), len(self.classes_)))
        oof_soul = np.zeros((len(X), len(self.classes_)))

        for t_ix, v_ix in skf.split(X_scaled, y):
            self.unit_logic.fit(X_scaled[t_ix], y[t_ix])
            self.unit_grad.fit(X_scaled[t_ix], y[t_ix])
            self.unit_soul.fit(X_scaled[t_ix], y[t_ix])
            oof_brain[v_ix] = self.unit_logic.predict_proba(X_scaled[v_ix])
            oof_grad[v_ix] = self.unit_grad.predict_proba(X_scaled[v_ix])
            oof_soul[v_ix] = self.unit_soul.predict_proba(X_scaled[v_ix])

        def loss_func(w):
            w = w / np.sum(w)
            blended = w[0]*oof_brain + w[1]*oof_grad + w[2]*oof_soul
            return log_loss(y, np.clip(blended, 1e-15, 1-1e-15))

        res = minimize(
            loss_func, [0.33, 0.33, 0.33],
            bounds=[(0, 1)]*3,
            constraints={'type': 'eq', 'fun': lambda w: 1 - sum(w)}
        )
        self.weights_ = res.x

        if self.verbose:
            print(f"  > G.O.D. Weights: Logic={self.weights_[0]:.2f} | "
                  f"Grad={self.weights_[1]:.2f} | Soul={self.weights_[2]:.2f}")
            print(f"  > Soul DNA: {self.unit_soul.dna_}")

        # Final Charge
        self.unit_logic.fit(X_scaled, y)
        self.unit_grad.fit(X_scaled, y)
        self.unit_soul.fit(X_scaled, y)
        return self

    def predict(self, X):
        X_scaled = self.scaler_.transform(X)
        p = (self.weights_[0]*self.unit_logic.predict_proba(X_scaled) +
             self.weights_[1]*self.unit_grad.predict_proba(X_scaled) +
             self.weights_[2]*self.unit_soul.predict_proba(X_scaled))
        return self.classes_[np.argmax(p, axis=1)]

def HarmonicResonanceForest_Ultimate(n_estimators=None):
    return HarmonicResonanceClassifier_GOD_v26(verbose=True)


✅ GPU DETECTED: HRF v26.0 'Holo-Fractal Universe' Active


In [11]:
from sklearn.datasets import fetch_openml
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

def run_comparative_benchmark(dataset_name, openml_id, sample_limit=3000):
    print(f"\n[DATASET] Loading {dataset_name} (ID: {openml_id})...")

    try:
        # Fetch as DataFrame to handle types better
        X, y = fetch_openml(data_id=openml_id, return_X_y=True, as_frame=True, parser='auto')

        # 1. AUTO-CLEANER: Convert Objects/Strings to Numbers
        for col in X.columns:
            if X[col].dtype == 'object' or X[col].dtype.name == 'category':
                le = LabelEncoder()
                X[col] = le.fit_transform(X[col].astype(str))

        X = X.values # Convert to Numpy for HRF

        # 2. Handle NaNs
        if np.isnan(X).any():
            from sklearn.impute import SimpleImputer
            imp = SimpleImputer(strategy='mean')
            X = imp.fit_transform(X)

        le_y = LabelEncoder()
        y = le_y.fit_transform(y)

        # 3. GPU Limit Check
        if len(X) > sample_limit:
            print(f"  ...Downsampling from {len(X)} to {sample_limit} (GPU Limit)...")
            X, y = resample(X, y, n_samples=sample_limit, random_state=42, stratify=y)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
        print(f"  Shape: {X.shape} | Classes: {len(np.unique(y))}")

    except Exception as e:
        print(f"  Error loading data: {e}")
        return

    competitors = {
        "SVM (RBF)": make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1.0, probability=True, random_state=42)),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
        "XGBoost (GPU)": XGBClassifier(
            device='cuda',
            tree_method='hist',
            use_label_encoder=False,
            eval_metric='logloss',
            random_state=42
        ),
        "HRF Ultimate (GPU)": HarmonicResonanceForest_Ultimate(n_estimators=60)
    }

    results = {}
    print(f"\n[BENCHMARK] Executing comparisons on {dataset_name}...")
    print("-" * 65)
    print(f"{'Model Name':<25} | {'Accuracy':<10} | {'Status'}")
    print("-" * 65)

    hrf_acc = 0

    for name, model in competitors.items():
        try:
            model.fit(X_train, y_train)
            preds = model.predict(X_test)
            acc = accuracy_score(y_test, preds)
            results[name] = acc
            print(f"{name:<25} | {acc:.4%}    | Done")

            if "HRF" in name:
                hrf_acc = acc

        except Exception as e:
            print(f"{name:<25} | FAILED      | {e}")

    print("-" * 65)

    best_competitor = 0
    for k, v in results.items():
        if "HRF" not in k and v > best_competitor:
            best_competitor = v

    margin = hrf_acc - best_competitor

    if margin > 0:
        print(f" HRF WINNING MARGIN: +{margin:.4%}")
    else:
        print(f" HRF GAP: {margin:.4%}")



In [12]:
# TEST 1: EEG Eye State
# ID: 1471
# Type: Biological Time-Series (Periodic)

run_comparative_benchmark(
    dataset_name="EEG Eye State",
    openml_id=1471,
    sample_limit=3000  # Fast Mode Active
)


[DATASET] Loading EEG Eye State (ID: 1471)...
  ...Downsampling from 14980 to 3000 (GPU Limit)...
  Shape: (3000, 14) | Classes: 2

[BENCHMARK] Executing comparisons on EEG Eye State...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 85.3333%    | Done
Random Forest             | 89.5000%    | Done
XGBoost (GPU)             | 89.5000%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 90.00% | Grad: 88.96% | Soul: 92.92%
  > G.O.D. Weights: Logic=0.00 | Grad=0.31 | Soul=0.69
  > Soul DNA: {'freq': 1.2245830004282552, 'gamma': 1.7764978440077908, 'power': 2.0, 'metric': 'minkowski', 'p': np.float64(1.8578683540723464), 'phase': 1.5615885385756962, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 93.5000%    | Done
-----------------------------------------------------------------
 HRF WINNIN

In [ ]:
# TEST 2: Phoneme
# ID: 1489
# Type: Audio/Harmonic Time-Series

run_comparative_benchmark(
    dataset_name="Phoneme",
    openml_id=1489,
    sample_limit=3000
)


[DATASET] Loading Phoneme (ID: 1489)...
  ...Downsampling from 5404 to 3000 (GPU Limit)...
  Shape: (3000, 5) | Classes: 2

[BENCHMARK] Executing comparisons on Phoneme...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 81.6667%    | Done
Random Forest             | 91.0000%    | Done
XGBoost (GPU)             | 91.5000%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 90.83% | Grad: 90.42% | Soul: 91.25%
  > G.O.D. Weights: Logic=0.43 | Grad=0.38 | Soul=0.19
  > Soul DNA: {'freq': 3.4367665557475573, 'gamma': 4.851766776833486, 'power': 3.0, 'metric': 'minkowski', 'p': np.float64(2.663280161150536), 'phase': 2.4021256222638208, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 92.8333%    | Done
-----------------------------------------------------------------
 HRF WINNING MARGIN: +1.333

In [ ]:
# TEST 3: Wall-Following Robot Navigation
# ID: 1497
# Type: Sensor/Geometric (Ultrasound Waves)

run_comparative_benchmark(
    dataset_name="Wall-Following Robot",
    openml_id=1497,
    sample_limit=3000
)


[DATASET] Loading Wall-Following Robot (ID: 1497)...
  ...Downsampling from 5456 to 3000 (GPU Limit)...
  Shape: (3000, 24) | Classes: 4

[BENCHMARK] Executing comparisons on Wall-Following Robot...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 88.5000%    | Done
Random Forest             | 99.5000%    | Done
XGBoost (GPU)             | 99.6667%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 97.50% | Grad: 99.79% | Soul: 95.00%
  > G.O.D. Weights: Logic=0.00 | Grad=0.99 | Soul=0.01
  > Soul DNA: {'freq': 0.9150493375641906, 'gamma': 1.142446013280745, 'power': 0.5, 'metric': 'minkowski', 'p': np.float64(0.6435406465092507), 'phase': 1.410130730484019, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 99.6667%    | Done
-----------------------------------------------------------------


In [ ]:
# TEST 4: Electricity
# ID: 151
# Type: Time-Series / Economic Flow (Periodic)

run_comparative_benchmark(
    dataset_name="Electricity",
    openml_id=151,
    sample_limit=3000
)


[DATASET] Loading Electricity (ID: 151)...
  ...Downsampling from 45312 to 3000 (GPU Limit)...
  Shape: (3000, 8) | Classes: 2

[BENCHMARK] Executing comparisons on Electricity...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 78.0000%    | Done
Random Forest             | 84.0000%    | Done
XGBoost (GPU)             | 83.1667%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 82.71% | Grad: 83.75% | Soul: 80.42%
  > G.O.D. Weights: Logic=0.48 | Grad=0.52 | Soul=0.00
  > Soul DNA: {'freq': 1.8015456812264756, 'gamma': 0.5449939100581996, 'power': 2.0, 'metric': 'minkowski', 'p': np.float64(1.5346249270611183), 'phase': 1.1243432860708815, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 85.3333%    | Done
-----------------------------------------------------------------
 HRF WINNING MARG

In [ ]:
# TEST 5: Gas Sensor Array Drift
# ID: 1476
# Type: Chemical Sensors / Physics (High Dimensional)

run_comparative_benchmark(
    dataset_name="Gas Sensor Drift",
    openml_id=1476,
    sample_limit=3000
)


[DATASET] Loading Gas Sensor Drift (ID: 1476)...
  ...Downsampling from 13910 to 3000 (GPU Limit)...
  Shape: (3000, 128) | Classes: 6

[BENCHMARK] Executing comparisons on Gas Sensor Drift...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 93.6667%    | Done
Random Forest             | 98.8333%    | Done
XGBoost (GPU)             | 98.1667%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 98.33% | Grad: 97.92% | Soul: 98.75%
  > G.O.D. Weights: Logic=0.10 | Grad=0.75 | Soul=0.15
  > Soul DNA: {'freq': 0.4980087859287794, 'gamma': 3.9105140110427272, 'power': 3.0, 'metric': 'minkowski', 'p': np.float64(1.766306790089581), 'phase': 2.4387486882673404, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 98.6667%    | Done
-----------------------------------------------------------------
 HRF 

In [ ]:
# TEST 6: Japanese Vowels
# ID: 375
# Type: Audio / Speech (Harmonic Time-Series)

run_comparative_benchmark(
    dataset_name="Japanese Vowels",
    openml_id=375,
    sample_limit=3000
)


[DATASET] Loading Japanese Vowels (ID: 375)...
  ...Downsampling from 9961 to 3000 (GPU Limit)...
  Shape: (3000, 14) | Classes: 9

[BENCHMARK] Executing comparisons on Japanese Vowels...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 97.8333%    | Done
Random Forest             | 94.3333%    | Done
XGBoost (GPU)             | 95.1667%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 95.62% | Grad: 94.58% | Soul: 95.62%
  > G.O.D. Weights: Logic=0.00 | Grad=0.37 | Soul=0.63
  > Soul DNA: {'freq': 1.668701123982972, 'gamma': 4.646730727786341, 'power': 2.0, 'metric': 'minkowski', 'p': np.float64(2.2079620572831944), 'phase': 0.16661552711366595, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 97.1667%    | Done
-----------------------------------------------------------------
 HRF GAP: 

In [ ]:
# TEST 7: Gesture Phase Segmentation
# ID: 4538
# Type: 3D Motion / Human Kinematics

run_comparative_benchmark(
    dataset_name="Gesture Phase",
    openml_id=4538,
    sample_limit=3000
)


[DATASET] Loading Gesture Phase (ID: 4538)...
  ...Downsampling from 9873 to 3000 (GPU Limit)...
  Shape: (3000, 32) | Classes: 5

[BENCHMARK] Executing comparisons on Gesture Phase...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 55.0000%    | Done
Random Forest             | 69.1667%    | Done
XGBoost (GPU)             | 67.8333%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 65.83% | Grad: 66.04% | Soul: 64.58%
  > G.O.D. Weights: Logic=0.55 | Grad=0.37 | Soul=0.08
  > Soul DNA: {'freq': 0.5270100750339822, 'gamma': 3.230248198500015, 'power': 2.0, 'metric': 'minkowski', 'p': np.float64(2.203158959469002), 'phase': 2.9127669816506443, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 69.5000%    | Done
-----------------------------------------------------------------
 HRF WINNING M

In [ ]:
# TEST 8: Mfeat-Fourier
# ID: 14
# Type: Geometric Frequencies / Fourier Coefficients
# Hypothesis: The "Soul" Unit should contain the highest weight here.

run_comparative_benchmark(
    dataset_name="Mfeat-Fourier",
    openml_id=14,
    sample_limit=3000
)


[DATASET] Loading Mfeat-Fourier (ID: 14)...
  Shape: (2000, 76) | Classes: 10

[BENCHMARK] Executing comparisons on Mfeat-Fourier...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 87.7500%    | Done
Random Forest             | 85.7500%    | Done
XGBoost (GPU)             | 87.2500%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 85.00% | Grad: 81.88% | Soul: 80.62%
  > G.O.D. Weights: Logic=0.30 | Grad=0.52 | Soul=0.17
  > Soul DNA: {'freq': 0.7724892221729361, 'gamma': 0.1053798724715708, 'power': 6.0, 'metric': 'minkowski', 'p': np.float64(2.4705456452254486), 'phase': 0.02746710679017204, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 88.0000%    | Done
-----------------------------------------------------------------
 HRF WINNING MARGIN: +0.2500%


In [ ]:
# TEST 9: Optdigits (Optical Recognition of Handwritten Digits)
# ID: 28
# Type: Image / Geometry
# Hypothesis: Handwriting is about Shape Flow, not Logic Rules. Soul should rise.

run_comparative_benchmark(
    dataset_name="Optdigits",
    openml_id=28,
    sample_limit=3000
)


[DATASET] Loading Optdigits (ID: 28)...
  ...Downsampling from 5620 to 3000 (GPU Limit)...
  Shape: (3000, 64) | Classes: 10

[BENCHMARK] Executing comparisons on Optdigits...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 99.0000%    | Done
Random Forest             | 99.1667%    | Done
XGBoost (GPU)             | 98.5000%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 98.54% | Grad: 97.50% | Soul: 98.33%
  > G.O.D. Weights: Logic=0.00 | Grad=0.59 | Soul=0.41
  > Soul DNA: {'freq': 0.5989072159193799, 'gamma': 0.5, 'power': 1.0, 'metric': 'minkowski', 'p': np.float64(1.687355858934798), 'phase': 0.11480645513552398, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 98.8333%    | Done
-----------------------------------------------------------------
 HRF GAP: -0.3333%


In [ ]:
# TEST 11: Texture Analysis (Kylberg)
# ID: 40975
# Type: Image Texture / Surface Physics
# Hypothesis: Texture is Frequency. Soul should dominate.

run_comparative_benchmark(
    dataset_name="Texture Analysis",
    openml_id=40975,
    sample_limit=3000
)


[DATASET] Loading Texture Analysis (ID: 40975)...
  Shape: (1728, 6) | Classes: 4

[BENCHMARK] Executing comparisons on Texture Analysis...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 90.4624%    | Done
Random Forest             | 98.2659%    | Done
XGBoost (GPU)             | 99.4220%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 97.47% | Grad: 99.64% | Soul: 93.50%
  > G.O.D. Weights: Logic=0.00 | Grad=1.00 | Soul=0.00
  > Soul DNA: {'freq': 1.8398547060061974, 'gamma': 0.11513601065358567, 'power': 2.0, 'metric': 'minkowski', 'p': np.float64(0.6249035025474244), 'phase': 0.14205687835313566, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 99.4220%    | Done
-----------------------------------------------------------------
 HRF GAP: 0.0000%


In [ ]:
# TEST 12: Steel Plates Faults
# ID: 1504
# Type: Industrial Physics / Surface Geometry
# Hypothesis: Defects are geometric shapes. Soul should assist.

run_comparative_benchmark(
    dataset_name="Steel Plates Faults",
    openml_id=1504,
    sample_limit=2000
)


[DATASET] Loading Steel Plates Faults (ID: 1504)...
  Shape: (1941, 33) | Classes: 2

[BENCHMARK] Executing comparisons on Steel Plates Faults...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 99.4859%    | Done
Random Forest             | 99.2288%    | Done
XGBoost (GPU)             | 100.0000%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 100.00% | Grad: 100.00% | Soul: 98.39%
  > G.O.D. Weights: Logic=1.00 | Grad=0.00 | Soul=0.00
  > Soul DNA: {'freq': 0.9596748207802431, 'gamma': 4.507233665289338, 'power': 6.0, 'metric': 'minkowski', 'p': np.float64(3.628192526266541), 'phase': 0.48461331279121855, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 100.0000%    | Done
-----------------------------------------------------------------
 HRF GAP: 0.0000%


In [21]:
# TEST 13: Climate Model Simulation Crashes
# ID: 1467
# Type: Chaos Theory / Atmospheric Physics
# Hypothesis: Chaos is just complex Resonance. Soul should wake up.

run_comparative_benchmark(
    dataset_name="Climate Model Crashes",
    openml_id=1467,
    sample_limit=3000
)


[DATASET] Loading Climate Model Crashes (ID: 1467)...
  Shape: (540, 20) | Classes: 2

[BENCHMARK] Executing comparisons on Climate Model Crashes...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 91.6667%    | Done
Random Forest             | 91.6667%    | Done
XGBoost (GPU)             | 92.5926%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 91.95% | Grad: 90.80% | Soul: 93.10%
  > G.O.D. Weights: Logic=0.88 | Grad=0.12 | Soul=0.00
  > Soul DNA: {'freq': 1.3681947921292528, 'gamma': 0.5, 'power': 2.0, 'metric': 'minkowski', 'p': np.float64(1.96895310719959), 'phase': 1.1631441365130106, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 91.6667%    | Done
-----------------------------------------------------------------
 HRF GAP: -0.9259%


In [ ]:
# TEST 14: Segment (Image Segmentation)
# ID: 1468
# Type: Visual Physics / Surface Classification
# Hypothesis: Texture = Frequency. Soul should dominate.

run_comparative_benchmark(
    dataset_name="Segment",
    openml_id=1468,
    sample_limit=3000
)


[DATASET] Loading Segment (ID: 1468)...
  Shape: (1080, 856) | Classes: 9

[BENCHMARK] Executing comparisons on Segment...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 88.8889%    | Done
Random Forest             | 90.7407%    | Done
XGBoost (GPU)             | 90.7407%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 93.06% | Grad: 90.75% | Soul: 88.44%
  > G.O.D. Weights: Logic=0.26 | Grad=0.74 | Soul=0.00
  > Soul DNA: {'freq': 0.9179093088676261, 'gamma': 0.41739099103731014, 'power': 6.0, 'metric': 'minkowski', 'p': np.float64(1.9917541519231594), 'phase': 2.6234920446907504, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 92.1296%    | Done
-----------------------------------------------------------------
 HRF WINNING MARGIN: +1.3889%


In [13]:
# TEST 15: Bioresponse (Molecular Activity)
# ID: 4134
# Type: Chemo-informatics / Molecular Physics
# Hypothesis: Molecular Activity is Resonance (Lock & Key).
#             High-Dim Holography is required.

run_comparative_benchmark(
    dataset_name="Bioresponse",
    openml_id=4134,
    sample_limit=1000
)


[DATASET] Loading Bioresponse (ID: 4134)...
  ...Downsampling from 3751 to 1000 (GPU Limit)...
  Shape: (1000, 1776) | Classes: 2

[BENCHMARK] Executing comparisons on Bioresponse...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 76.5000%    | Done
Random Forest             | 82.0000%    | Done
XGBoost (GPU)             | 80.0000%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 81.88% | Grad: 82.50% | Soul: 80.62%
  > G.O.D. Weights: Logic=0.60 | Grad=0.40 | Soul=0.00
  > Soul DNA: {'freq': 0.09028959550076471, 'gamma': 0.5, 'power': 1.0, 'metric': 'minkowski', 'p': np.float64(2.1280319866250093), 'phase': 2.7404593108627555, 'dim_reduction': 'pca'}
HRF Ultimate (GPU)        | 84.5000%    | Done
-----------------------------------------------------------------
 HRF WINNING MARGIN: +2.5000%

In [14]:
# TEST 16: Higgs Boson (Particle Physics)
# ID: 23512
# Type: High Energy Physics / Subatomic Kinetics
# Hypothesis: Particle decay follows quantum resonance patterns.
#             The Soul should vibrate with the Higgs field.

run_comparative_benchmark(
    dataset_name="Higgs Boson",
    openml_id=23512,
    sample_limit=3000
)


[DATASET] Loading Higgs Boson (ID: 23512)...
  ...Downsampling from 98050 to 3000 (GPU Limit)...
  Shape: (3000, 28) | Classes: 2

[BENCHMARK] Executing comparisons on Higgs Boson...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 66.5000%    | Done
Random Forest             | 68.6667%    | Done
XGBoost (GPU)             | 66.6667%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 67.71% | Grad: 65.00% | Soul: 61.46%
  > G.O.D. Weights: Logic=0.70 | Grad=0.30 | Soul=0.00
  > Soul DNA: {'freq': 0.8530796115694949, 'gamma': 0.5, 'power': 0.5, 'metric': 'minkowski', 'p': 2.0, 'phase': 0.2667405105613437, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 69.1667%    | Done
-----------------------------------------------------------------
 HRF WINNING MARGIN: +0.5000%


In [15]:
# TEST 17: Magic Gamma Telescope (Astrophysics)
# ID: 1120
# Type: Astrophysics / Cherenkov Radiation
# Hypothesis: Gamma showers create specific geometric ellipses.
#             Pure geometry = Soul territory.

run_comparative_benchmark(
    dataset_name="Magic Telescope",
    openml_id=1120,
    sample_limit=3000
)


[DATASET] Loading Magic Telescope (ID: 1120)...
  ...Downsampling from 19020 to 3000 (GPU Limit)...
  Shape: (3000, 10) | Classes: 2

[BENCHMARK] Executing comparisons on Magic Telescope...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 86.3333%    | Done
Random Forest             | 88.3333%    | Done
XGBoost (GPU)             | 87.6667%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 88.75% | Grad: 87.92% | Soul: 83.96%
  > G.O.D. Weights: Logic=0.54 | Grad=0.46 | Soul=0.00
  > Soul DNA: {'freq': 1.290875466312706, 'gamma': 4.915282803212547, 'power': 3.0, 'metric': 'minkowski', 'p': np.float64(2.2643418270363354), 'phase': 1.1300255279328153, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 89.0000%    | Done
-----------------------------------------------------------------
 HRF WINN

In [16]:
# TEST 18: Musk v2 (Biochemistry)
# ID: 1116
# Type: Chemo-informatics / Molecular Shape
# Hypothesis: Olfactory perception is based on molecular vibration (Turin's Theory).
#             This is the ultimate test for Harmonic Resonance.

run_comparative_benchmark(
    dataset_name="Musk v2",
    openml_id=1116,
    sample_limit=3000
)


[DATASET] Loading Musk v2 (ID: 1116)...
  ...Downsampling from 6598 to 3000 (GPU Limit)...
  Shape: (3000, 167) | Classes: 2

[BENCHMARK] Executing comparisons on Musk v2...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 99.6667%    | Done
Random Forest             | 99.8333%    | Done
XGBoost (GPU)             | 100.0000%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 98.54% | Grad: 100.00% | Soul: 96.67%
  > G.O.D. Weights: Logic=0.00 | Grad=1.00 | Soul=0.00
  > Soul DNA: {'freq': 0.2564816724071525, 'gamma': 0.717097830158589, 'power': 0.5, 'metric': 'minkowski', 'p': np.float64(1.908247756164599), 'phase': 1.0620019232404252, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 100.0000%    | Done
-----------------------------------------------------------------
 HRF GAP: 0.0000%


In [17]:
# TEST 19: Satellite Image (Satimage)
# ID: 182
# Type: Remote Sensing / Spectral Physics
# Hypothesis: Soil and vegetation emit specific spectral frequencies.
#             The Soul's frequency analysis should separate them easily.

run_comparative_benchmark(
    dataset_name="Satimage",
    openml_id=182,
    sample_limit=3000
)


[DATASET] Loading Satimage (ID: 182)...
  ...Downsampling from 6430 to 3000 (GPU Limit)...
  Shape: (3000, 36) | Classes: 6

[BENCHMARK] Executing comparisons on Satimage...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 88.1667%    | Done
Random Forest             | 93.6667%    | Done
XGBoost (GPU)             | 93.0000%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 93.54% | Grad: 93.12% | Soul: 93.33%
  > G.O.D. Weights: Logic=0.26 | Grad=0.43 | Soul=0.31
  > Soul DNA: {'freq': 1.2619792048382843, 'gamma': 3.912157561373206, 'power': 2.0, 'metric': 'minkowski', 'p': np.float64(2.1073116849061546), 'phase': 1.615614465607031, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 93.6667%    | Done
-----------------------------------------------------------------
 HRF GAP: 0.0000%


In [18]:
# TEST 20: Letter Recognition (Computer Vision)
# ID: 6
# Type: Geometric Pattern Recognition
# Hypothesis: Letters are defined by curves and relative distances.
#             Distance-based models (Soul) usually beat Trees here.

run_comparative_benchmark(
    dataset_name="Letter Recognition",
    openml_id=6,
    sample_limit=3000
)


[DATASET] Loading Letter Recognition (ID: 6)...
  ...Downsampling from 20000 to 3000 (GPU Limit)...
  Shape: (3000, 16) | Classes: 26

[BENCHMARK] Executing comparisons on Letter Recognition...
-----------------------------------------------------------------
Model Name                | Accuracy   | Status
-----------------------------------------------------------------
SVM (RBF)                 | 86.3333%    | Done
Random Forest             | 91.3333%    | Done
XGBoost (GPU)             | 89.1667%    | Done
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 89.17% | Grad: 88.12% | Soul: 85.42%
  > G.O.D. Weights: Logic=0.07 | Grad=0.58 | Soul=0.35
  > Soul DNA: {'freq': 1.3507139077435901, 'gamma': 3.508472690560883, 'power': 6.0, 'metric': 'minkowski', 'p': np.float64(1.345125219317291), 'phase': 0.0, 'dim_reduction': 'none'}
HRF Ultimate (GPU)        | 91.8333%    | Done
-----------------------------------------------------------------
 HRF WINNING MARGIN:

In [20]:
import joblib
import os
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder

# --- 1. HELPER: DATA LOADER (Restored) ---
def load_openml_dataset(openml_id, sample_limit=3000):
    print(f"  > Fetching ID: {openml_id} from OpenML...")
    try:
        # Fetch data
        dataset = fetch_openml(data_id=openml_id, as_frame=False, parser='auto')
        X = dataset.data
        y = dataset.target

        # Handle Categorical Targets
        if y.dtype == 'object' or isinstance(y[0], str):
            le = LabelEncoder()
            y = le.fit_transform(y)

        # Handle NaN in X (Simple Imputation for stability)
        if np.isnan(X).any():
            from sklearn.impute import SimpleImputer
            imp = SimpleImputer(strategy='median')
            X = imp.fit_transform(X)

        # Downsample if needed (to respect GPU/Time limits)
        if len(X) > sample_limit:
            print(f"  > Downsampling from {len(X)} to {sample_limit}...")
            X, y = resample(X, y, n_samples=sample_limit, stratify=y, random_state=42)

        return X, y, dataset.details.get('name', 'Unknown'), "Classification"

    except Exception as e:
        print(f"❌ Error loading ID {openml_id}: {e}")
        return None, None, None, None

# --- 2. CRYOSTASIS SYSTEM (Save/Load) ---

def save_god_model(model, filename="HRF_Ultimate_Gen1.pkl"):
    """Saves the trained G.O.D. model to disk."""
    if model is None:
        print("❌ Error: No model to save!")
        return

    print(f"❄️ Initiating Cryostasis for {filename}...")
    joblib.dump(model, filename)
    file_size = os.path.getsize(filename) / (1024 * 1024)
    print(f"✅ G.O.D. Model Saved Successfully! (Size: {file_size:.2f} MB)")
    print(f"   path: {os.path.abspath(filename)}")

def load_god_model(filename="HRF_Ultimate_Gen1.pkl"):
    """Awakens the G.O.D. model from disk."""
    if not os.path.exists(filename):
        print(f"❌ Error: File {filename} not found.")
        return None

    print(f"⚡ Awakening G.O.D. from {filename}...")
    model = joblib.load(filename)
    print("✅ System Online. Ready for Inference.")
    return model

# --- 3. EXECUTION ---
print("\n[TRAINING FINAL MODEL FOR EXPORT]")
# Re-training on Letter Recog (ID 6) as the flagship example
# You can change ID to 4134 (Bioresponse) or any other winning dataset
X_final, y_final, name, _ = load_openml_dataset(6, 3000)

if X_final is not None:
    god_model = HarmonicResonanceForest_Ultimate()
    god_model.fit(X_final, y_final)

    # Save it
    save_god_model(god_model, "HRF_Ultimate_v26_LetterRecog.pkl")

    # Verify it works
    print("\n[VERIFICATION TEST]")
    loaded_god = load_god_model("HRF_Ultimate_v26_LetterRecog.pkl")
    sample_data = X_final[:5]
    predictions = loaded_god.predict(sample_data)
    print(f"🔮 Predictions from Loaded Model: {predictions}")
    print(f"🎯 Actual Labels: {y_final[:5]}")


[TRAINING FINAL MODEL FOR EXPORT]
  > Fetching ID: 6 from OpenML...
  > Downsampling from 20000 to 3000...
  > Initiating G.O.D. v26 (Holo-Fractal Universe)...
    [Evolution] Logic: 92.17% | Grad: 90.17% | Soul: 89.67%
  > G.O.D. Weights: Logic=0.02 | Grad=0.62 | Soul=0.35
  > Soul DNA: {'freq': 1.4316439850377303, 'gamma': 1.7441176904577416, 'power': 3.0, 'metric': 'minkowski', 'p': np.float64(1.3504327579161035), 'phase': 0.0, 'dim_reduction': 'none'}
❄️ Initiating Cryostasis for HRF_Ultimate_v26_LetterRecog.pkl...
✅ G.O.D. Model Saved Successfully! (Size: 114.20 MB)
   path: /content/HRF_Ultimate_v26_LetterRecog.pkl

[VERIFICATION TEST]
⚡ Awakening G.O.D. from HRF_Ultimate_v26_LetterRecog.pkl...
✅ System Online. Ready for Inference.
🔮 Predictions from Loaded Model: [ 0  5 21 20 24]
🎯 Actual Labels: [ 0  5 21 20 24]
